In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"MPS available: {torch.backends.mps.is_available()}")

# Building a Generative Diffusion Model with WoW Icons

## Project Objectives

In this notebook, I'll document the process of building a generative diffusion model that can create new WoW-style icons (kind of).

This implementation uses a simple DDPM (Denoising Diffusion Probabilistic Model) approach. The model exclusively removes noise based on random input noise images, without any conditioning or optimizations. In fact, it's entirely unoptimized for running on my laptop's limited hardware, so this notebook is really a learning exercise to help me understand the fundamentals of diffusion models.

## Process Overview

<img src="assets/Diffusion%20Model%20Pytorch.webp" alt="Diffusion Model PyTorch" style="width:700px; height:auto;"><br>
<i>(['Understanding Deep Learning'](https://mitpress.mit.edu/9780262048644/understanding-deep-learning/) by Simon J.D. Prince via ['Diffusion Model from Scratch in Pytorch'](https://towardsdatascience.com/diffusion-model-from-scratch-in-pytorch-ddpm-9d9760528946/) by Nicholas DiSalvo)</i>

Diffusion models work by gradually adding noise to images (forward process) and then learning to reverse this process (reverse process), allowing for high-quality image generation.

The forward process is a predetermined 'Markov chain' based on a noise schedule. Whereby; "predetermined" means we decide in advance exactly how much noise to add at each step according to our noise schedule - it's not random but follows a specific pattern, and "Markov chain" means each noisy version only depends on the previous step - like a chain where each link only connects to the ones right next to it. When you combine these concepts, you can precisely calculate how noisy an image will be at any step in the process, setting the stage for a model to learn how to remove the noise that has been added.

<img src="assets/unet-model-diagram.webp" alt="Unet Model" style="width:500px; height:auto;"><br>
<i>(UNET for Diffusion by [Nicholas DiSalvo](https://towardsdatascience.com/author/nickd16718/))</i>

A U-Net architecture is trained to predict the noise that was added at any arbitrary timestep, by taking a noisy image at a specific timestep and trying to predict what noise was added to it.<br>The U-Net architecture itself, with its downsampling path (reduces spatial dimensions) and upsampling (increases spatial dimensions) path connected by skip connections, is particularly good at this task because:

- The downsampling captures the overall structure and context
- The upsampling reconstructs detailed features
- The skip connections help preserve important information from earlier layers

It could be optimised with scoring or conditional inputs, but we're keeping things simple for now.

## Dataset Information

For this project, I'm using the "CleanIcons-MechagnomeEdition" dataset, specifically version 11.1.0.59347-V4-1, which contains a collection of World of Warcraft icons (upscaled by Mechagnome).
This dataset provides a foundation for exploring diffusion model training on stylistically consistent gaming assets - It may genuinely not work at all, but let's see...

### Dataset Details

- **Name**: CleanIcons-MechagnomeEdition-11.1.0.59347-V4-1
- **Source**: [GitHub - AcidWeb/Clean-Icons-Mechagnome-Edition](https://github.com/AcidWeb/Clean-Icons-Mechagnome-Edition/releases/tag/11.1.0.59347-V4-1)
- **Contents**: World of Warcraft icons
- **Format**: TGA (Truevision Graphics Adapter) image files
- **Features**: High-quality game icons with transparency support

### Implementation Notes

This implementation adapts the diffusion model architecture to work with:

- TGA format images (using PIL for loading)
- Apple Silicon hardware (ensuring MPS device compatibility)
- Custom data loading pipeline for WoW icon processing

### Credits and References

This implementation takes direct reference from ["A Diffusion Model from Scratch in PyTorch"](https://colab.research.google.com/drive/1sjy9odlSSy0RBVgMTgP7s99NXsqglsUL?usp=sharing#scrollTo=Rj17psVw7Shg).

The original work references:

- GitHub implementation of [Denoising Diffusion PyTorch](https://github.com/lucidrains/denoising-diffusion-pytorch)
- Work by Niels Rogge and Kashif Rasul in their [Huggingface notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=3a159023)
- Academic papers on Diffusion models:
  - [Improved Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2102.09672) (Nichol & Dhariwal, 2021)
  - [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) (Ho et al., 2020)
  - [Diffusion Models Beat GANs on Image Synthesis](https://arxiv.org/abs/2105.05233) (Dhariwal & Nichol, 2021)

I've adapted this implementation to work specifically with TGA format WoW icons on Apple Silicon hardware, modifying the data loading pipeline and ensuring MPS device compatibility.


In [ ]:
# display random samples from dataset
fig = plt.figure(figsize=(10, 10))

icons_list = os.listdir("./data/ICONS")
icons_list_sample = np.random.choice(icons_list, size=15)

for i, item in enumerate(icons_list_sample):
    icon_path = os.path.join("./data/ICONS", item)
    icon_image = Image.open(icon_path)
    plt.subplot(5, 5, i + 1)
    plt.imshow(icon_image)
    plt.axis("off")

fig.suptitle("Random Samples from Dataset", fontsize=16)
plt.tight_layout()
plt.show()

## Forward Diffusion Process Explained

The forward diffusion process gradually adds noise to an image over a series of timesteps. Let's break down the key components:

### Noise Schedule Function

```python
def linear_beta_schedule(timesteps, start=0.0001, end=0.02):
    return torch.linspace(start, end, timesteps)
```

This function creates a linear schedule for the noise variance (β) at each timestep:

- Starts with minimal noise (`start=0.0001`)
- Gradually increases to larger noise (`end=0.02`)
- Creates `timesteps` evenly spaced values
- The schedule controls how quickly information is destroyed during diffusion

### Helper Function for Batch Processing

```python
def get_index_from_list(vals, t, x_shape):
    """
    Returns a specific index t of a passed list of values vals
    while considering the batch dimension.
    """
    batch_size = t.shape[0]
    out = vals.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)
```

This utility function:

- Extracts specific values from a parameter tensor based on timestep indices
- Reshapes the output to properly broadcast across the image dimensions
- Ensures values are on the correct device for computation
- Enables efficient batch processing of images at different timesteps

### Forward Diffusion Function

```python
def forward_diffusion_sample(x_0, t, device="mps"):
    """
    Takes an image and a timestep as input and
    returns the noisy version of it
    """
    noise = torch.randn_like(x_0)
    sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x_0.shape
    )
    # mean + variance
    return sqrt_alphas_cumprod_t.to(device) * x_0.to(
        device
    ) + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)
```

This implements the core diffusion equation from Ho et al. (2020):

$$q(x_t|x_0) = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1-\bar{\alpha}_t} \cdot \epsilon$$

Where:

- $x_0$ is the original image
- $x_t$ is the noisy image at timestep $t$
- $\epsilon$ is random Gaussian noise
- $\bar{\alpha}_t$ represents how much of the original signal remains at timestep $t$

### Pre-calculated Diffusion Parameters

```python
# Define beta schedule
T = 300
betas = linear_beta_schedule(timesteps=T)

# Pre-calculate different terms for closed form
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)
posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)
```

These calculations:

- Set up the full noise schedule for all timesteps
- Pre-compute constants needed for both forward diffusion and the eventual reverse process
- Define how the signal-to-noise ratio changes at each step of the diffusion
- Calculate the posterior variance needed for the sampling process

The mathematical setup follows the DDPM paper (Ho et al., 2020), where:

- $\alpha_t = 1 - \beta_t$
- $\bar{\alpha}_t = \prod_{i=1}^{t} \alpha_i$

These pre-calculations allow for efficient training and sampling as we don't need to recompute these values repeatedly.


In [4]:
import torch.nn.functional as F


def linear_beta_schedule(timesteps, start=0.0001, end=0.02):
    return torch.linspace(start, end, timesteps)


def get_index_from_list(vals, t, x_shape):
    """
    Returns a specific index t of a passed list of values vals
    while considering the batch dimension.
    """
    batch_size = t.shape[0]
    out = vals.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)


def forward_diffusion_sample(x_0, t, device="mps"):
    """
    Takes an image and a timestep as input and
    returns the noisy version of it
    """
    noise = torch.randn_like(x_0)
    sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x_0.shape
    )
    # mean + variance
    return sqrt_alphas_cumprod_t.to(device) * x_0.to(
        device
    ) + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)


# Define beta schedule
T = 300
betas = linear_beta_schedule(timesteps=T)

# Pre-calculate different terms for closed form
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)
posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)

## Dataset and Data Loading Pipeline

This section sets up the data loading pipeline for our WoW icons dataset. <br>
It handles loading the TGA image files, preprocessing them, and creating batches for training.


In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Check for MPS availability
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

IMG_SIZE = 16
BATCH_SIZE = 256


# Custom transforms that can be pickled (unlike lambda functions)
class NormalizeNegOneToOne(object):
    def __call__(self, x):
        return (x * 2) - 1

    def __repr__(self):
        return self.__class__.__name__ + "()"


class PermuteCHWToHWC(object):
    def __call__(self, x):
        return x.permute(1, 2, 0)

    def __repr__(self):
        return self.__class__.__name__ + "()"


class ToNumpy(object):
    def __call__(self, x):
        return x.numpy().astype(np.uint8)

    def __repr__(self):
        return self.__class__.__name__ + "()"


class WoWIconsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with the TGA files
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = [f for f in os.listdir(root_dir) if f.endswith(".tga")]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_list[idx])
        # PIL can open TGA files
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image


def load_transformed_dataset(data_dir):
    data_transforms = [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),  # Scales data into [0,1]
        NormalizeNegOneToOne(),  # Scale between [-1, 1]
    ]
    data_transform = transforms.Compose(data_transforms)

    dataset = WoWIconsDataset(root_dir=data_dir, transform=data_transform)
    return dataset


def show_tensor_image(image):
    # First move tensor to CPU before transformations
    if image.device.type != "cpu":
        image = image.cpu()

    reverse_transforms = transforms.Compose(
        [
            transforms.Lambda(
                lambda t: (t + 1) / 2
            ),  # Safe to use Lambda here as it's not in DataLoader
            PermuteCHWToHWC(),  # CHW to HWC
            transforms.Lambda(
                lambda t: t * 255.0
            ),  # Safe to use Lambda here as it's not in DataLoader
            ToNumpy(),
            transforms.ToPILImage(),
        ]
    )

    # Take first image of batch
    if len(image.shape) == 4:
        image = image[0, :, :, :]
    plt.imshow(reverse_transforms(image))


# Assuming your extracted TGA files are in a directory called "wow_icons"
data_dir = "./data/ICONS"
data = load_transformed_dataset(data_dir)

# Configure DataLoader with zero workers for now to avoid pickle issues
dataloader = DataLoader(
    data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=0,  # Use single process for now to avoid pickle errors
)


# Function to move batch to device efficiently
def prepare_batch(batch, target_device=device):
    return batch.to(target_device)

## Forward Diffusion Demo

This visualization displays the forward diffusion process applied to our WoW icons. Each row shows a different randomly selected icon from the dataset, while columns represent progressive timesteps in the diffusion process.<br><br>
Moving from left to right along each row, you see how the diffusion model gradually adds more noise to the original clean images. The leftmost column shows the original, unmodified WoW icons in their clean state. As you move right, each subsequent image shows the same icon with increasing amounts of random noise applied according to a carefully controlled schedule (T=300).<br><br>
By the rightmost column, the icons have been almost completely transformed into pure noise, with little to no recognizable features remaining from the original images. This visual demonstration illustrates how the forward diffusion process systematically destroys information in the original images over time.<br><br>
Understanding this forward process is crucial because training a diffusion model involves teaching a neural network to reverse this exact process - starting from pure noise and progressively removing it to generate new, realistic WoW icons.


In [ ]:
def display_icons_with_diffusion(
    dataloader, num_icons=3, num_steps=7, T=300, device="mps"
):
    """
    Display a grid of randomly selected icons with the diffusion process applied.
    Each row shows one icon progressively diffused with noise from left to right.
    The first column shows the original, unmodified icon.

    Args:
        dataloader: PyTorch dataloader containing the icon dataset
        num_icons: Number of different icons to display (rows)
        num_steps: Number of diffusion steps to display (not including original)
        T: Total number of timesteps in the diffusion process
        device: Device to run computations on
    """
    # Get a batch of images
    batch = next(iter(dataloader))

    # Move batch to device
    batch = prepare_batch(batch, device)

    # Create figure
    plt.figure(figsize=((num_steps + 1) * 2, num_icons * 2))

    # Get random indices from the batch
    batch_size = batch.shape[0]
    indices = np.random.choice(batch_size, size=num_icons, replace=False)

    # Calculate the step size between diffusion steps
    stepsize = int(T / num_steps)

    # Plot each selected image with different noise levels
    for row, idx in enumerate(indices):
        # Get the original image
        original_img = batch[idx].unsqueeze(0)  # Add batch dimension

        # First, display the original image with no noise
        plt.subplot(num_icons, num_steps + 1, row * (num_steps + 1) + 1)
        plt.axis("off")

        if original_img.device.type != "cpu":
            original_display = original_img.cpu()
        else:
            original_display = original_img

        show_tensor_image(original_display)

        # Add "Original" as title for the first row
        if row == 0:
            plt.title("Original", fontsize=8)

        # Apply different levels of noise for the remaining columns
        for col, timestep in enumerate(range(0, T, stepsize)):
            if col >= num_steps:
                break

            plt.subplot(
                num_icons, num_steps + 1, row * (num_steps + 1) + col + 2
            )  # +2 because we already used column 1
            plt.axis("off")

            # Apply forward diffusion with current timestep
            t = torch.tensor([timestep], device=device).type(torch.int64)
            noisy_img, _ = forward_diffusion_sample(original_img, t, device=device)

            # Display the image
            if noisy_img.device.type != "cpu":
                noisy_img = noisy_img.cpu()

            show_tensor_image(noisy_img)

            # Add timestep as title for the first row
            if row == 0:
                plt.title(f"t={timestep}", fontsize=8)

    plt.tight_layout()
    plt.show()


# Usage example
display_icons_with_diffusion(dataloader, num_icons=3, num_steps=7, T=T)

## The Reverse Diffusion Process and U-Net Architecture

While the forward diffusion process gradually adds noise to images, the reverse diffusion process is where the magic bits happens - <br>It's how we generate new images by progressively removing noise. This is accomplished using a neural network (in our case, a U-Net) that predicts the noise component at each timestep.

### The Reverse Diffusion Equation

The diffusion model learns to reverse the forward process by predicting the noise ε that was added at each timestep. The mathematical equation for the reverse process is:

$$p_\theta(x_{t-1}|x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \Sigma_\theta(x_t, t))$$

Where:

- $p_\theta$ is our model's reverse distribution
- $\mu_\theta$ is the predicted mean
- $\Sigma_\theta$ is the predicted variance

Instead of trying to learn how to generate images directly (which is very hard), the neural network (U-Net) is trained on a simpler task: "Given an image with noise at step t, can you predict what noise was added?". Once the network is trained to predict noise accurately, we can use it to generate brand new images by starting with pure random noise and then iteratively "removing" the predicted noise.

### U-Net Architecture Overview

A U-Net is a specific type of neural network architecture that was originally developed for biomedical image segmentation but has become widely used across many image processing tasks, including diffusion models.
The mechanism that makes U-Net effective for both medical image segmentation and diffusion models comes down to some key architectural properties that transfer well between these seemingly different tasks:

- <b>Preserving spatial relationships</b>: In medical imaging, U-Net excels at identifying boundaries and structures while maintaining their spatial relationships. In diffusion models, this same ability helps preserve the underlying structure of an image while noise is being predicted or removed.
- <b>Multi-scale feature processing</b>: Medical images often contain features at different scales (from tiny cell structures to larger organs). Similarly, noise in diffusion models affects features at multiple scales - from fine textures to broader shapes. The U-Net's downsampling/upsampling path <i>(reducing/increasing the spatial dimensions of an image or feature map.)</i> processes information at various resolutions, making it effective for both tasks.
- <b>Context integration with precision</b>: For medical segmentation, U-Net combines broad contextual information (Is this a liver? A brain?) with precise boundaries. For diffusion, it balances understanding the overall image structure while precisely identifying noise patterns at each pixel location.
- <b>Transformation precision</b>: Both tasks require pixel-level precision in predicting output images. Whether defining the exact boundary of a tumor or determining exactly how much noise is at each pixel location, the U-Net's architecture is designed for this precise image-to-image mapping.
- <b>Skip connections</b>: In a U-Net, a 'skip connection' directly connects a layer in the downsampling path to its corresponding layer in the upsampling path. For example, the features from the first downsampling layer are connected to the last upsampling layer, the second downsampling layer to the second-to-last upsampling layer, and so on. Skip connections solve a fundamental problem in deep networks: a) The downsampling path is great for capturing context but loses spatial precision b) The upsampling path needs to reconstruct spatial details but starts from compressed information
  - In medical imaging: They help preserve fine boundary details that would be lost in downsampling
  - In diffusion models: They help the model maintain the underlying image structure while focusing on noise prediction

The fundamental similarity is that both tasks involve transforming one image into another while preserving important structural information - whether that's transforming a medical scan into a segmentation map or transforming a noisy image into a noise prediction (which then allows for denoising).

Our implementation uses a simplified U-Net with:

- A downsampling path that reduces spatial dimensions and increases feature channels
- An upsampling path that increases spatial dimensions and decreases feature channels
- Skip connections that concatenate features from the downsampling path to the upsampling path
- Time embeddings that inform the network which diffusion timestep it's operating on

### Time Embeddings

A critical aspect of diffusion models is that the network needs to know which timestep it's denoising. We use sinusoidal position embeddings, similar to those in transformer architectures:

```python
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings
```

These embeddings create unique representations for each timestep that the model can use to apply the appropriate amount of denoising.

### Building Blocks

The `Block` class forms the foundation of both the downsampling and upsampling paths:

```python
class Block(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)
        if up:
            self.conv1 = nn.Conv2d(2*in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose2d(out_ch, out_ch, 4, 2, 1)
        else:
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv2d(out_ch, out_ch, 4, 2, 1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)
        self.bnorm2 = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()
```

Each block:

- Takes in a feature map and timestep embedding
- Processes the feature map with convolutions and nonlinearities
- Injects the timestep information via addition
- Either downsamples (using strided convolution) or upsamples (using transposed convolution)

### Complete U-Net Architecture

The `SimpleUnet` class brings everything together:

```python
class SimpleUnet(nn.Module):
    def __init__(self):
        super().__init__()
        image_channels = 3
        down_channels = (64, 128, 256, 512, 1024)
        up_channels = (1024, 512, 256, 128, 64)
        out_dim = 3
        time_emb_dim = 32

        # Time embedding
        self.time_mlp = nn.Sequential(
                SinusoidalPositionEmbeddings(time_emb_dim),
                nn.Linear(time_emb_dim, time_emb_dim),
                nn.ReLU()
            )

        # Initial projection
        self.conv0 = nn.Conv2d(image_channels, down_channels[0], 3, padding=1)

        # Downsample
        self.downs = nn.ModuleList([Block(down_channels[i], down_channels[i+1], \
                                    time_emb_dim) \
                    for i in range(len(down_channels)-1)])
        # Upsample
        self.ups = nn.ModuleList([Block(up_channels[i], up_channels[i+1], \
                                        time_emb_dim, up=True) \
                    for i in range(len(up_channels)-1)])

        self.output = nn.Conv2d(up_channels[-1], out_dim, 1)
```

The network:

1. Takes an image and timestep as input
2. Embeds the timestep using sinusoidal embeddings
3. Processes the image through a series of downsampling blocks, reducing spatial dimensions and capturing increasingly abstract features
4. Processes through upsampling blocks, gradually increasing spatial dimensions while incorporating features from the corresponding downsampling blocks via skip connections
5. Produces an output with the same spatial dimensions and channels as the input image, representing the predicted noise

### Training Objective

The model is trained to predict the noise that was added during the forward diffusion process. The loss function is simply the mean squared error between the predicted noise and the actual noise:

$$L = \mathbb{E}_{t,x_0,\epsilon}[||\epsilon - \epsilon_\theta(x_t, t)||^2]$$

Where:

- $\epsilon$ is the actual noise added
- $\epsilon_\theta$ is the model's prediction of the noise
- $x_t$ is the noisy image at timestep $t$

By minimizing this loss, the model learns to estimate the noise component at any given timestep, which is the key to generating new images through the reverse diffusion process.


In [ ]:
import torch
from torch import nn
import math


class Block(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)
        if up:
            self.conv1 = nn.Conv2d(2 * in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose2d(out_ch, out_ch, 4, 2, 1)
        else:
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv2d(out_ch, out_ch, 4, 2, 1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)
        self.bnorm2 = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()

    def forward(self, x, t):
        # First Conv
        h = self.bnorm1(self.relu(self.conv1(x)))
        # Time embedding
        time_emb = self.relu(self.time_mlp(t))
        # Extend last 2 dimensions
        time_emb = time_emb[(...,) + (None,) * 2]
        # Add time channel
        h = h + time_emb
        # Second Conv
        h = self.bnorm2(self.relu(self.conv2(h)))
        # Down or Upsample
        return self.transform(h)


class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


class SimpleUnet(nn.Module):
    """
    A simplified variant of the Unet architecture for M1 Pro compatibility.
    """

    def __init__(self, device="mps"):
        super().__init__()
        self.device = device
        image_channels = 4
        down_channels = (64, 128, 256, 512, 1024)
        up_channels = (1024, 512, 256, 128, 64)
        out_dim = 3
        time_emb_dim = 32

        # Time embedding
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.ReLU(),
        )

        # Initial projection
        self.conv0 = nn.Conv2d(image_channels, down_channels[0], 3, padding=1)

        # Downsample
        self.downs = nn.ModuleList(
            [
                Block(down_channels[i], down_channels[i + 1], time_emb_dim)
                for i in range(len(down_channels) - 1)
            ]
        )
        # Upsample
        self.ups = nn.ModuleList(
            [
                Block(up_channels[i], up_channels[i + 1], time_emb_dim, up=True)
                for i in range(len(up_channels) - 1)
            ]
        )

        self.output = nn.Conv2d(up_channels[-1], out_dim, 1)

        # Move model to appropriate device
        self.to(device)

    def forward(self, x, timestep):
        # Ensure inputs are on the correct device
        x = x.to(self.device)
        timestep = timestep.to(self.device)

        # Embedd time
        t = self.time_mlp(timestep)
        # Initial conv
        x = self.conv0(x)
        # Unet
        residual_inputs = []
        for down in self.downs:
            x = down(x, t)
            residual_inputs.append(x)
        for up in self.ups:
            residual_x = residual_inputs.pop()
            # Add residual x as additional channels
            x = torch.cat((x, residual_x), dim=1)
            x = up(x, t)
        return self.output(x)


def get_loss(model, x_0, t, device="mps"):
    """
    Calculates training loss for a batch of images at specified timesteps.
    """
    x_0 = x_0.to(device)
    t = t.to(device)

    # Get noise
    noise = torch.randn_like(x_0).to(device)

    # Get noisy image at timestep t
    x_noisy, _ = forward_diffusion_sample(x_0, t, device)

    # Get predicted noise from model
    noise_pred = model(x_noisy, t)

    # Calculate loss (mean squared error between actual and predicted noise)
    loss = F.mse_loss(noise, noise_pred)

    return loss


# Initialize model
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = SimpleUnet(device=device)
print(f"Using device: {device}")
print(f"Num params: {sum(p.numel() for p in model.parameters())}")

# Setup optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model

## Loss Function

"Loss" refers to a mathematical measure of how well your diffusion model is performing.
It quantifies the error or difference between what your model predicts and the actual target values.
<br>For your diffusion model specifically:

1. You start with a clean image (a WoW icon)
2. You add a known amount of random noise to it
3. You ask your model to predict what noise was added
4. The "loss" measures how far off your model's prediction is from the actual noise that was added

The F.l1_loss() function calculates the Mean Absolute Error (MAE), which is the average of the absolute differences between the predicted noise and the actual noise. A smaller loss value means your model is doing a better job at predicting the noise.<br>
During training, your optimization algorithm (like Adam) uses this loss value to update the model's parameters, gradually improving its ability to predict the noise. The goal is to minimize this loss over many training iterations.<br>
Once the model is trained to accurately predict the noise at each timestep, you can use it in reverse - starting with pure noise and gradually removing the predicted noise components to generate new WoW icons that never existed before.


In [8]:
def get_loss(model, x_0, t):
    """
    Calculates training loss for a batch of images at specified timesteps.
    Handles channel dimension mismatch between noise and model output.
    """
    # Create a noisy version of the original image at timestep t
    x_noisy, noise = forward_diffusion_sample(x_0, t, device)

    # Use the model to predict what noise was added
    noise_pred = model(x_noisy, t)

    # Adjust noise channels to match prediction channels if needed
    if noise.shape[1] != noise_pred.shape[1]:
        # Keep only the first 3 channels of noise to match noise_pred
        noise = noise[:, : noise_pred.shape[1], :, :]

    # Calculate the L1 loss (mean absolute error) between
    # the actual noise and the predicted noise
    return F.l1_loss(noise, noise_pred)

## Sampling

### Key Implementation Details

- These functions implement the sampling/generation process for creating new images with our diffusion model.

- **Memory Management**: Without adding `@torch.no_grad()` we quickly run out of memory, because PyTorch tracks all the previous images for gradient calculation. When you use `@torch.no_grad()` as a decorator for your sampling functions, you're telling PyTorch not to track operations for automatic differentiation (gradients).

- **Noise Schedule Consistency**: During the reverse process (image generation), we need to know exactly how much noise to remove at each step. For this to work properly, we need to use the same noise schedule that we used when adding noise. It's like following a recipe in reverse - you need to know the exact amounts that were added originally to remove them correctly.

### Benefits of Pre-calculated Values

The pre-calculated values ensure that:

1. We're removing the right amount of noise at each step
2. We're adding the right amount of new randomness when needed (yes, sometimes we add a small amount of new noise during generation)

If we didn't use these pre-calculated values and instead made up new values during the reverse process, the math wouldn't work out correctly, and we'd get blurry or unrealistic images.


In [9]:
@torch.no_grad()
def sample_timestep(x, t):
    """
    This function performs a single step of the reverse diffusion process,
    taking a noisy image and making it slightly less noisy.
    Calls the model to predict the noise in the image and returns
    the denoised image. Applies noise to this image, if we are not in the last step yet.

        1. Gets the appropriate noise parameters for the current timestep
        2. Uses your trained model to predict what noise is present in the current noisy image
        3. Uses that prediction to calculate a cleaner version of the image
        4. If we're not at the final step, adds a small amount of new random noise (helps with sample quality)
        5. Returns the slightly cleaner image
    """
    betas_t = get_index_from_list(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = get_index_from_list(sqrt_recip_alphas, t, x.shape)

    # Get model prediction (the model outputs only 3 channels)
    model_output = model(x, t)

    # Handle channel mismatch: pad model output to match input if needed
    if model_output.shape[1] != x.shape[1]:
        # Create a tensor of zeros with the same shape as x
        padded_output = torch.zeros_like(x)
        # Copy the available channels from model_output
        padded_output[:, : model_output.shape[1]] = model_output
        model_output = padded_output

    # Call model (current image - noise prediction)
    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model_output / sqrt_one_minus_alphas_cumprod_t
    )
    posterior_variance_t = get_index_from_list(posterior_variance, t, x.shape)

    if t == 0:
        # As pointed out by Luis Pereira (see YouTube comment)
        # The t's are offset from the t's in the paper
        return model_mean
    else:
        noise = torch.randn_like(x)
        return model_mean + torch.sqrt(posterior_variance_t) * noise


@torch.no_grad()
def sample_plot_image():
    """
    This function generates a completely new image by iteratively applying
    the sampling process and visualizing the intermediate steps.
        1. Starts with completely random noise (a new "seed" image)
        2. Sets up a figure to display the generation process
        3. Iterates through all timesteps in reverse order (from T down to 0)
        4. At each step, calls sample_timestep to make the image a bit cleaner
        5. Clamps the pixel values to keep them in the valid range
        6. Periodically visualizes the current state to show the progressive denoising
    """
    # Sample noise with 4 channels to match model's expected input
    img_size = IMG_SIZE
    img = torch.randn(
        (1, 4, img_size, img_size), device=device
    )  # Changed from 3 to 4 channels
    plt.figure(figsize=(15, 2))
    plt.axis("off")
    num_images = 10
    stepsize = int(T / num_images)

    for i in range(0, T)[::-1]:
        t = torch.full((1,), i, device=device, dtype=torch.long)
        img = sample_timestep(img, t)
        # Edit: This is to maintain the natural range of the distribution
        img = torch.clamp(img, -1.0, 1.0)
        if i % stepsize == 0:
            plt.subplot(1, num_images, int(i / stepsize) + 1)
            # For display, we may need to use only the first 3 channels if showing RGB
            display_img = img
            if img.shape[1] > 3:
                display_img = img[:, :3]  # Use only first 3 channels for display
            show_tensor_image(display_img.detach().cpu())
    plt.show()

In [ ]:
import os
import time
import torch
from torch.optim import Adam

# Check for MPS (Metal Performance Shaders) for Mac with Apple Silicon
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA")
else:
    device = "cpu"
    print("Using CPU")

# Create a directory for checkpoints if it doesn't exist
checkpoint_dir = "diffusion_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)


def save_checkpoint(epoch, model, optimizer, loss, filename=None):
    """
    Save training checkpoint
    """
    if filename is None:
        filename = f"{checkpoint_dir}/checkpoint_epoch_{epoch}.pt"

    print(f"Saving checkpoint to {filename}")
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": loss,
        },
        filename,
    )

    # Also save a 'latest' checkpoint for easy resuming
    latest_path = f"{checkpoint_dir}/checkpoint_latest.pt"
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": loss,
        },
        latest_path,
    )


def load_checkpoint(model, optimizer, filename=None):
    """
    Load a training checkpoint
    """
    if filename is None:
        filename = f"{checkpoint_dir}/checkpoint_latest.pt"

    if not os.path.exists(filename):
        print(f"No checkpoint found at {filename}")
        return 0, 0

    print(f"Loading checkpoint from {filename}")
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    return checkpoint["epoch"], checkpoint["loss"]


model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)

# Training configuration
epochs = 75
checkpoint_frequency = 10  # Save a checkpoint every 10 epochs
checkpoint_time_interval = 30 * 60  # Also save every 30 minutes
resume_training = True  # Set to True to resume from latest checkpoint

# Try to load previous checkpoint if resuming training
start_epoch = 0
if resume_training:
    start_epoch, _ = load_checkpoint(model, optimizer)

# Time of last checkpoint
last_checkpoint_time = time.time()

# Training loop
for epoch in range(start_epoch, epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    epoch_loss = 0
    batch_count = 0

    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device).long()
        loss = get_loss(model, batch, t)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        batch_count += 1

        # Print progress
        if step % 25 == 0:
            print(f"Step {step} of {len(dataloader)}, Loss: {loss.item():.6f}")

        # Time-based checkpoint
        current_time = time.time()
        if current_time - last_checkpoint_time > checkpoint_time_interval:
            save_checkpoint(
                epoch,
                model,
                optimizer,
                loss.item(),
                f"{checkpoint_dir}/checkpoint_time_{int(current_time)}.pt",
            )
            last_checkpoint_time = current_time

    # Calculate average epoch loss
    avg_epoch_loss = epoch_loss / batch_count
    print(f"Epoch {epoch+1} completed. Average loss: {avg_epoch_loss:.6f}")

    # Visualization
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Generating sample image after epoch {epoch+1}")
        sample_plot_image()

    # Epoch-based checkpoint
    if (epoch + 1) % checkpoint_frequency == 0:
        save_checkpoint(epoch + 1, model, optimizer, avg_epoch_loss)

# Save final model
save_checkpoint(
    epochs, model, optimizer, avg_epoch_loss, f"{checkpoint_dir}/model_final.pt"
)
print("Training completed!")

# Save a standalone model for inference (just the weights, no optimizer state)
torch.save(model.state_dict(), "diffusion_model_final.pth")
print("Saved standalone model weights to diffusion_model_final.pth")

# Conclusion

So .. Unfortunately my macbook isn't really optmised for this sort of model training 😅, in that, it would take me forever and I'm unable to practically optimise it (at least for this data set).
I'm going to try the same approach but with the MNIST dataset, just to get an idea of how it works - Watch this space!
